### Power simulation

Test of comparative statics

In [1]:
import matplotlib.pyplot as plt 
import statsmodels.stats.power as smp 
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from itertools import product 


In [2]:
def compute_test_result(n_matching_grp_per_treatment, 
                        n_obs_per_matching_grp,
                        small_coal_worth, 
                        unit, 
                        err_type, 
                        err_scale, 
                        ind_set, 
                        use_treatment_dummies, 
                        cov_type):
    """
    Run a statistical test on data generated by assumed DGP

    Parameters: 
    - experiment set-up
        * n_matching_grp_per_treatment # number of matching groups per treatment/session
        * n_obs_per_matching_grp # number of observations per matching group
        * small_coal_worth # list of treatment values
    - DGP:
        * err_type = "uniform" # choose: "uniform", "norm_corr"
        * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
        * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
    - specification:
        * use_treatment_dummies # choose: 0,1
        * unit # choose: "group", "matching_group"
        * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3" (if using group as a unit)
    
    Returns:
    - p value(s) for coefficients on treatment dummies / worth of small coalition
    """
    # Generate independent variables
    
    n_treat = len(small_coal_worth)
    n_obs = n_obs_per_matching_grp * n_matching_grp_per_treatment * n_treat # total number of group observations
    
    Xval = np.array(sum([n_obs_per_matching_grp * n_matching_grp_per_treatment * [val] for val in small_coal_worth], [])) 
    X = pd.get_dummies(Xval, drop_first= True).values.astype(int) if use_treatment_dummies else Xval 
    X = sm.add_constant(X)

    if unit == "matching_group" or (unit == "group" and cov_type == "cluster"): 
        clusters = np.array(sum([n_obs_per_matching_grp * [cluster] for cluster in range(0, n_matching_grp_per_treatment * n_treat)], [])) 
        X = np.column_stack((X, clusters))

    # Assumed DGP 

    if err_type == "uniform":
        e = np.random.random(n_obs) * 2 * err_scale - err_scale
    elif err_type == "norm_corr": # generate correlated errors within each matching group
        cov = err_scale * np.random.random(size=(n_obs_per_matching_grp,n_obs_per_matching_grp))
        cov = cov + cov.T # make it symmetric 
        cov = np.dot(cov, cov.T) # make it positive semidefinite
        e = multivariate_normal.rvs(mean = np.zeros(n_obs_per_matching_grp), cov = cov, size = n_matching_grp_per_treatment * n_treat).flatten()
        
    es = np.ones(n_obs) * (100//3)  
    sh = (100 + Xval)//3  
    no_coord = np.zeros(n_obs)
    ind = {0: es, 1: sh, 2: no_coord} 

    rand = [np.random.choice(ind_set) for j in range(n_obs)] 
    y = np.array([ind[rand[i]][i] for i in range(n_obs)]) + np.array([0 if rand[i]==2 else e[i] for i in range(n_obs)])
    y = np.maximum(y,0) 
    y = np.minimum(y,100)

    if unit == "matching_group":
        y = np.array([np.mean(y[X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])
        X = np.array([np.mean(X[:,0:-1][X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])

    # Run a statistical test (only on outcomes with successful coordination)

    y_reg = y[y>0] 
    X_reg = X[y>0]

    if unit == "matching_group":
        results = sm.regression.linear_model.OLS(y_reg, X_reg).fit() 
    elif unit == "group":
        if cov_type == "cluster":
            results = sm.regression.linear_model.OLS(y_reg, X_reg[:, :-1]).fit(cov_type = cov_type, cov_kwds= {"groups": X_reg[:, -1]}) # clustered standard errors
        else:
            results = sm.regression.linear_model.OLS(y_reg, X_reg).fit(cov_type = cov_type) # robust standard errors

    return (results.pvalues[1], results.pvalues[2]) if use_treatment_dummies else results.pvalues[1]


In [3]:
def power_simulation(num_runs, 
                     sig_level, 
                     n_matching_grp_per_treatment, 
                     n_obs_per_matching_grp, 
                     small_coal_worth, 
                     unit, 
                     err_type, 
                     err_scale, 
                     ind_set, 
                     use_treatment_dummies, 
                     cov_type):
    """
        Run a power simulation for our bargaining experiment 
    
        Parameters: 
        - simulation:
            * num_runs 
            * sig_level 
        - experiment set-up
            * n_matching_grp_per_treatment # number of matching groups per treatment/session
            * n_obs_per_matching_grp # number of observations per matching group
            * small_coal_worth # list of treatment values
            * unit # choose: "group", "matching_group"
        - DGP:
            * err_type = "uniform" # choose: "uniform", "norm_corr"
            * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
            * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
        - specification:
            * use_treatment_dummies # choose: 0,1
            * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3"
        
        Returns: 
        - Dataframe with p value(s) for each simulation run.
    """
    power_sim_results = pd.DataFrame({'p_value_1': np.zeros(num_runs), 'p_value_2': np.zeros(num_runs)}) if use_treatment_dummies else pd.DataFrame({'p_value_1': np.zeros(num_runs)}) 

    for run in range(num_runs):
        power_sim_results.loc[run, :] = compute_test_result(n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit, err_type, err_scale, ind_set, use_treatment_dummies, cov_type)

    #print([f"{key}: {value}" for key, value in locals().items() if key not in ["power_sim_results", "run"]])
    power_sim_summary = dict([(key,value) for key, value in locals().items() if key not in ["power_sim_results", "run"]])

    p_reject_1 = np.mean(power_sim_results['p_value_1'] < sig_level)
    #print(f"Simulated power for treatment dummy 1: {p_reject_1} for significance level {sig_level}") 
    if use_treatment_dummies: 
        p_reject_2 = np.mean(power_sim_results['p_value_2'] < sig_level)
        #print(f"Simulated power for treatment dummy 2: {p_reject_2} for significance level {sig_level}") 

    power_sim_summary["power_coeff1"] = p_reject_1 
    power_sim_summary["power_coeff2"] = p_reject_2 if use_treatment_dummies else np.nan

    return power_sim_summary


In [4]:
# run power simulation over a grid of parameters
num_runs = [5000]
sig_level = [0.05]
n_matching_grp_per_treatment = [5]
n_obs_per_matching_grp = [10]
small_coal_worth = [[10,30,90]]
unit = ["group", "matching_group"]
err_type = ["uniform", "norm_corr"] 
err_scale =  [2,3,5]
ind_set = [[0,0,0,1,1,1,2], [0,1,1,1,1,2]]
use_treatment_dummies = [0,1]
cov_type = ["cluster", "HC3"]

res = pd.DataFrame()

combinations = list(product(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit, err_type, err_scale, ind_set, use_treatment_dummies, cov_type))
for comb in combinations: 
    print(comb)
    r = pd.DataFrame.from_dict(power_simulation(*comb), orient = "index").transpose()
    res = pd.concat([res, r], axis = 0)

(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 0, 0, 1, 1, 1, 2], 0, 'cluster')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 0, 0, 1, 1, 1, 2], 0, 'HC3')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 0, 0, 1, 1, 1, 2], 1, 'cluster')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 0, 0, 1, 1, 1, 2], 1, 'HC3')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 1, 1, 1, 1, 2], 0, 'cluster')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 1, 1, 1, 1, 2], 0, 'HC3')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 1, 1, 1, 1, 2], 1, 'cluster')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 2, [0, 1, 1, 1, 1, 2], 1, 'HC3')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 3, [0, 0, 0, 1, 1, 1, 2], 0, 'cluster')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 3, [0, 0, 0, 1, 1, 1, 2], 0, 'HC3')
(5000, 0.05, 5, 10, [10, 30, 90], 'group', 'uniform', 3, [0, 0, 0, 1, 1, 1, 2], 1, 'cluster'

Basic analysis of results
* cov_type: cluster vs HC3
* unit: group vs matching group
* test: parametric vs nonparametric

In [6]:
# make them hashable
res["small_coal_worth"] = res["small_coal_worth"].apply(lambda x: tuple(x))
res["ind_set"] = res["ind_set"].apply(lambda x: tuple(x))

In [28]:
res_pivot = res.pivot(index = ["err_type", "err_scale", "ind_set"], columns = ["unit", "cov_type", "use_treatment_dummies"], values = ["power_coeff1", "power_coeff2"])
res_pivot

power_coeff1                  \
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                0       0       1   
err_type  err_scale ind_set                                              
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)       0.8416  0.9584  0.2056   
                    (0, 1, 1, 1, 1, 2)           0.997  0.9996   0.376   
          3         (0, 0, 0, 1, 1, 1, 2)        0.624  0.8644   0.154   
                    (0, 1, 1, 1, 1, 2)          0.9344   0.994   0.239   
          5         (0, 0, 0, 1, 1, 1, 2)       0.3536  0.7238  0.1214   
                    (0, 1, 1, 1, 1, 2)          0.6454  0.9174  0.1546   
uniform   2         (0, 0, 0, 1, 1, 1, 2)       0.9996  0.9998  0.9864   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)       0.9992     1.0  0.9766   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          5         (0, 0, 0, 1, 1, 1, 2)       0.9992  0.9998  0.9402   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0  0.9998   

                                                                          \
unit                                              matching_group           
cov_type                                      HC3        cluster     HC3   
use_treatment_dummies                           1              0       0   
err_type  err_scale ind_set                                                
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)   0.597         0.6246  0.6346   
                    (0, 1, 1, 1, 1, 2)     0.7592         0.9112  0.9184   
          3         (0, 0, 0, 1, 1, 1, 2)  0.5502         0.4324  0.4488   
                    (0, 1, 1, 1, 1, 2)     0.6528         0.7732  0.7874   
          5         (0, 0, 0, 1, 1, 1, 2)  0.5152         0.2252  0.2348   
                    (0, 1, 1, 1, 1, 2)     0.5476         0.4638  0.4632   
uniform   2         (0, 0, 0, 1, 1, 1, 2)  0.9878         0.8918  0.8898   
                    (0, 1, 1, 1, 1, 2)        1.0           0.99  0.9874   
          3         (0, 0, 0, 1, 1, 1, 2)   0.983         0.8954  0.8854   
                    (0, 1, 1, 1, 1, 2)        1.0         0.9894  0.9892   
          5         (0, 0, 0, 1, 1, 1, 2)  0.9334         0.8882  0.8766   
                    (0, 1, 1, 1, 1, 2)        1.0         0.9886  0.9894   

                                                          power_coeff2       \
unit                                                             group        
cov_type                                  cluster     HC3      cluster  HC3   
use_treatment_dummies                           1       1            0    0   
err_type  err_scale ind_set                                                   
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)   0.072  0.0708          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1218  0.1054          NaN  NaN   
          3         (0, 0, 0, 1, 1, 1, 2)  0.0658  0.0588          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.0868  0.0908          NaN  NaN   
          5         (0, 0, 0, 1, 1, 1, 2)   0.054  0.0542          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)      0.073  0.0646          NaN  NaN   
uniform   2         (0, 0, 0, 1, 1, 1, 2)   0.091  0.1024          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1828  0.1958          NaN  NaN   
          3         (0, 0, 0, 1, 1, 1, 2)  0.0982  0.0924          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1846  0.1994          NaN  NaN   
          5         (0, 0, 0, 1, 1, 1, 2)   0.093  0.0922          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1848  0.1918          NaN  NaN   

                                                                               \
unit                                                      matchin

In [86]:
num_cases = len(res_pivot[('power_coeff2','group', 'cluster', 1)] ) # num of DGPs

In [85]:
# comparison of cov_type: cluster vs HC3 (when unit == group) == > more power with HC3
print( (res_pivot[('power_coeff1','group', 'cluster', 0)] <= res_pivot[('power_coeff1', 'group', 'HC3', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','group', 'cluster', 1)] <= res_pivot[('power_coeff2', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','group', 'cluster', 1)] <= res_pivot[('power_coeff1', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

1.0
0.9166666666666666
1.0


In [108]:
res_pivot[res_pivot[('power_coeff1','group', 'cluster', 0)] <= res_pivot[('power_coeff1', 'group', 'HC3', 0)]]

power_coeff1                  \
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                0       0       1   
err_type  err_scale ind_set                                              
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)       0.8416  0.9584  0.2056   
                    (0, 1, 1, 1, 1, 2)           0.997  0.9996   0.376   
          3         (0, 0, 0, 1, 1, 1, 2)        0.624  0.8644   0.154   
                    (0, 1, 1, 1, 1, 2)          0.9344   0.994   0.239   
          5         (0, 0, 0, 1, 1, 1, 2)       0.3536  0.7238  0.1214   
                    (0, 1, 1, 1, 1, 2)          0.6454  0.9174  0.1546   
uniform   2         (0, 0, 0, 1, 1, 1, 2)       0.9996  0.9998  0.9864   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)       0.9992     1.0  0.9766   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          5         (0, 0, 0, 1, 1, 1, 2)       0.9992  0.9998  0.9402   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0  0.9998   

                                                                          \
unit                                              matching_group           
cov_type                                      HC3        cluster     HC3   
use_treatment_dummies                           1              0       0   
err_type  err_scale ind_set                                                
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)   0.597         0.6246  0.6346   
                    (0, 1, 1, 1, 1, 2)     0.7592         0.9112  0.9184   
          3         (0, 0, 0, 1, 1, 1, 2)  0.5502         0.4324  0.4488   
                    (0, 1, 1, 1, 1, 2)     0.6528         0.7732  0.7874   
          5         (0, 0, 0, 1, 1, 1, 2)  0.5152         0.2252  0.2348   
                    (0, 1, 1, 1, 1, 2)     0.5476         0.4638  0.4632   
uniform   2         (0, 0, 0, 1, 1, 1, 2)  0.9878         0.8918  0.8898   
                    (0, 1, 1, 1, 1, 2)        1.0           0.99  0.9874   
          3         (0, 0, 0, 1, 1, 1, 2)   0.983         0.8954  0.8854   
                    (0, 1, 1, 1, 1, 2)        1.0         0.9894  0.9892   
          5         (0, 0, 0, 1, 1, 1, 2)  0.9334         0.8882  0.8766   
                    (0, 1, 1, 1, 1, 2)        1.0         0.9886  0.9894   

                                                          power_coeff2       \
unit                                                             group        
cov_type                                  cluster     HC3      cluster  HC3   
use_treatment_dummies                           1       1            0    0   
err_type  err_scale ind_set                                                   
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)   0.072  0.0708          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1218  0.1054          NaN  NaN   
          3         (0, 0, 0, 1, 1, 1, 2)  0.0658  0.0588          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.0868  0.0908          NaN  NaN   
          5         (0, 0, 0, 1, 1, 1, 2)   0.054  0.0542          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)      0.073  0.0646          NaN  NaN   
uniform   2         (0, 0, 0, 1, 1, 1, 2)   0.091  0.1024          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1828  0.1958          NaN  NaN   
          3         (0, 0, 0, 1, 1, 1, 2)  0.0982  0.0924          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1846  0.1994          NaN  NaN   
          5         (0, 0, 0, 1, 1, 1, 2)   0.093  0.0922          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1848  0.1918          NaN  NaN   

                                                                               \
unit                                                      matchin

In [97]:
# comparison of unit: matching group vs group = > more power with group
print( (res_pivot[('power_coeff1','matching_group', 'cluster', 0)]  <= res_pivot[('power_coeff1', 'group', 'cluster', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','matching_group', 'HC3', 0)]      <= res_pivot[('power_coeff1', 'group', 'HC3', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','matching_group', 'cluster', 1)]  <= res_pivot[('power_coeff2', 'group', 'cluster', 1)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','matching_group', 'HC3', 1)]      <= res_pivot[('power_coeff2', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','matching_group', 'cluster', 1)]  <= res_pivot[('power_coeff1', 'group', 'cluster', 1)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','matching_group', 'HC3', 1)]      <= res_pivot[('power_coeff1', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

# note that theoretically the value for ('power_coeff1', 'matching_group', 'cluster', 0) should be the same as for ('power_coeff1', 'matching_group', 'HC3', 0) (because cov_type does not play a role for unit == matching_group), but here they are slightly different because of random sampling

1.0
1.0
1.0
1.0
1.0
1.0


In [98]:
res_pivot[res_pivot[('power_coeff1','matching_group', 'cluster', 0)] <= res_pivot[('power_coeff1', 'group', 'cluster', 0)]]

power_coeff1                  \
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                0       0       1   
err_type  err_scale ind_set                                              
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)       0.8416  0.9584  0.2056   
                    (0, 1, 1, 1, 1, 2)           0.997  0.9996   0.376   
          3         (0, 0, 0, 1, 1, 1, 2)        0.624  0.8644   0.154   
                    (0, 1, 1, 1, 1, 2)          0.9344   0.994   0.239   
          5         (0, 0, 0, 1, 1, 1, 2)       0.3536  0.7238  0.1214   
                    (0, 1, 1, 1, 1, 2)          0.6454  0.9174  0.1546   
uniform   2         (0, 0, 0, 1, 1, 1, 2)       0.9996  0.9998  0.9864   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)       0.9992     1.0  0.9766   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          5         (0, 0, 0, 1, 1, 1, 2)       0.9992  0.9998  0.9402   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0  0.9998   

                                                                          \
unit                                              matching_group           
cov_type                                      HC3        cluster     HC3   
use_treatment_dummies                           1              0       0   
err_type  err_scale ind_set                                                
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)   0.597         0.6246  0.6346   
                    (0, 1, 1, 1, 1, 2)     0.7592         0.9112  0.9184   
          3         (0, 0, 0, 1, 1, 1, 2)  0.5502         0.4324  0.4488   
                    (0, 1, 1, 1, 1, 2)     0.6528         0.7732  0.7874   
          5         (0, 0, 0, 1, 1, 1, 2)  0.5152         0.2252  0.2348   
                    (0, 1, 1, 1, 1, 2)     0.5476         0.4638  0.4632   
uniform   2         (0, 0, 0, 1, 1, 1, 2)  0.9878         0.8918  0.8898   
                    (0, 1, 1, 1, 1, 2)        1.0           0.99  0.9874   
          3         (0, 0, 0, 1, 1, 1, 2)   0.983         0.8954  0.8854   
                    (0, 1, 1, 1, 1, 2)        1.0         0.9894  0.9892   
          5         (0, 0, 0, 1, 1, 1, 2)  0.9334         0.8882  0.8766   
                    (0, 1, 1, 1, 1, 2)        1.0         0.9886  0.9894   

                                                          power_coeff2       \
unit                                                             group        
cov_type                                  cluster     HC3      cluster  HC3   
use_treatment_dummies                           1       1            0    0   
err_type  err_scale ind_set                                                   
norm_corr 2         (0, 0, 0, 1, 1, 1, 2)   0.072  0.0708          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1218  0.1054          NaN  NaN   
          3         (0, 0, 0, 1, 1, 1, 2)  0.0658  0.0588          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.0868  0.0908          NaN  NaN   
          5         (0, 0, 0, 1, 1, 1, 2)   0.054  0.0542          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)      0.073  0.0646          NaN  NaN   
uniform   2         (0, 0, 0, 1, 1, 1, 2)   0.091  0.1024          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1828  0.1958          NaN  NaN   
          3         (0, 0, 0, 1, 1, 1, 2)  0.0982  0.0924          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1846  0.1994          NaN  NaN   
          5         (0, 0, 0, 1, 1, 1, 2)   0.093  0.0922          NaN  NaN   
                    (0, 1, 1, 1, 1, 2)     0.1848  0.1918          NaN  NaN   

                                                                               \
unit                                                      matchin

In [ ]:
# comparison of tests: parametric vs nonparametric

In [106]:
# share of DGPs in grid with power >= 0.8, 0.9 
print(((res_pivot >= 0.8).astype(int).sum().sum())/(res_pivot == res_pivot).astype(int).sum().sum())
print(((res_pivot >= 0.9).astype(int).sum().sum())/(res_pivot == res_pivot).astype(int).sum().sum())

0.5555555555555556
0.4375


Test rejection rate when null is true/ type I error

In [111]:
# test rejection rate when null is true
num_runs = [5000]
sig_level = [0.05]
n_matching_grp_per_treatment = [5]
n_obs_per_matching_grp = [10]
small_coal_worth = [[10,30,90]]
unit = ["group", "matching_group"]
err_type = ["uniform", "norm_corr"] 
err_scale =  [2,3,5]
ind_set = [[0]]
use_treatment_dummies = [0,1]
cov_type = ["cluster", "HC3"]

res_null = pd.DataFrame()

combinations = list(product(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit, err_type, err_scale, ind_set, use_treatment_dummies, cov_type))
for comb in combinations: 
    #print(comb)
    r = pd.DataFrame.from_dict(power_simulation(*comb), orient = "index").transpose()
    res_null = pd.concat([res_null, r], axis = 0)

In [112]:
res_null["small_coal_worth"] = res_null["small_coal_worth"].apply(lambda x: tuple(x))
res_null["ind_set"] = res_null["ind_set"].apply(lambda x: tuple(x))

res_null_pivot = res_null.pivot(index = ["err_type", "err_scale", "ind_set"], columns = ["unit", "cov_type", "use_treatment_dummies"], values = ["power_coeff1", "power_coeff2"])
res_null_pivot

power_coeff1                          \
unit                               group                           
cov_type                         cluster     HC3 cluster     HC3   
use_treatment_dummies                  0       0       1       1   
err_type  err_scale ind_set                                        
norm_corr 2         (0,)             0.1  0.5356  0.1132  0.5392   
          3         (0,)          0.1036  0.5348  0.1034  0.5438   
          5         (0,)           0.097  0.5362  0.1078  0.5372   
uniform   2         (0,)          0.1004   0.055  0.1008  0.0458   
          3         (0,)          0.0928  0.0524  0.1034  0.0482   
          5         (0,)          0.0984  0.0548  0.1042  0.0496   

                                                                    \
unit                        matching_group                           
cov_type                           cluster     HC3 cluster     HC3   
use_treatment_dummies                    0       0       1       1   
err_type  err_scale ind_set                                          
norm_corr 2         (0,)            0.0526  0.0528  0.0508  0.0468   
          3         (0,)            0.0544  0.0524   0.054  0.0514   
          5         (0,)            0.0446  0.0512   0.054  0.0524   
uniform   2         (0,)            0.0548  0.0566  0.0472  0.0486   
          3         (0,)            0.0486  0.0568  0.0506   0.051   
          5         (0,)            0.0546  0.0496  0.0538  0.0472   

                            power_coeff2                                      \
unit                               group                      matching_group   
cov_type                         cluster  HC3 cluster     HC3        cluster   
use_treatment_dummies                  0    0       1       1              0   
err_type  err_scale ind_set                                                    
norm_corr 2         (0,)             NaN  NaN  0.1118  0.5558            NaN   
          3         (0,)             NaN  NaN   0.094  0.5448            NaN   
          5         (0,)             NaN  NaN  0.1042   0.536            NaN   
uniform   2         (0,)             NaN  NaN  0.1074   0.047            NaN   
          3         (0,)             NaN  NaN  0.1086   0.049            NaN   
          5         (0,)             NaN  NaN  0.1052  0.0538            NaN   

                                                  
unit                                              
cov_type                     HC3 cluster     HC3  
use_treatment_dummies          0       1       1  
err_type  err_scale ind_set                       
norm_corr 2         (0,)     NaN  0.0492  0.0438  
          3         (0,)     NaN  0.0506  0.0444  
          5         (0,)     NaN  0.0526  0.0536  
uniform   2         (0,)     NaN  0.0514  0.0484  
          3         (0,)     NaN  0.0504  0.0476  
          5         (0,)     NaN  0.0496   0.051